## Load cad+disease file

In [1]:
import os
import numpy as np
import pandas as pd
import json


label_json = pd.read_json("results/cad_disease.json")
label_json.head()

,study_id,subject_id,dicom_id,view,study_order,entity
0,55088298,18936629,61976388-5e534624-f6465079-76ea9caf-116f9938,antero-posterior,5,"{'pneumothorax': {'id': 0, 'entity_name': 'pne..."
1,51480826,18936629,ee88e702-2a3f531e-874e0bc0-e4e469e8-b34ed57a,antero-posterior,11,"{'pleural effusion': {'id': 0, 'entity_name': ..."
2,51268226,18936629,159014a8-5252eca6-b3c13717-20610f8d-ff5d2103,antero-posterior,24,{}
3,58729526,18936629,58c1653b-5d091bf0-ce12c7b3-7f6c738d-cd6f4e1b,postero-anterior,14,"{'pneumothorax': {'id': 0, 'entity_name': 'pne..."
4,56018384,18936629,a8becc3c-98292acf-114ca160-bb1e226a-fa031457,antero-posterior,32,"{'lung opacity': {'id': 0, 'entity_name': 'lun..."


In [2]:
print(len(label_json))
print("study_id: ", len(label_json["study_id"].unique()))
print("subject_id: ", len(label_json["subject_id"].unique()))
print("dicom_id: ", len(label_json["dicom_id"].unique()))

227827
study_id:  227827
subject_id:  65379
dicom_id:  227827


In [5]:
label_json["view"].unique()

array(['antero-posterior', 'postero-anterior', None, 'Erect', 'lateral',
       'left lateral', 'Recumbent', 'left anterior oblique'], dtype=object)

study_id: 227827. subject_id: 65379.

There are several kinds of view of CXR images in the file. We want to filter those images with AP and PA view.

In [3]:
df_ap_pa = pd.read_csv("files/mimic_ap_pa.csv")
df_ap_pa.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,...,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,path,split,report_path
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,files/p10/p10000032/s50414267/02aa804e-bde0afd...,train,files/p10/p10000032/s50414267.txt
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,train,files/p10/p10000032/s53189527.txt
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,files/p10/p10000032/s53911762/68b5c4b1-227d048...,train,files/p10/p10000032/s53911762.txt
3,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2906,2258,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,Erect,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,train,files/p10/p10000032/s53911762.txt
4,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800805,234424.765,CHEST (PORTABLE AP),antero-posterior,NaN,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,train,files/p10/p10000032/s56699142.txt


In [4]:
print(df_ap_pa.columns)
print(df_ap_pa["ViewPosition"].unique())

Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices',
       'dicom_id', 'PerformedProcedureStepDescription', 'ViewPosition', 'Rows',
       'Columns', 'StudyDate', 'StudyTime',
       'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'path', 'split',
       'report_path'],
      dtype='object')
['PA' 'AP']


In [5]:
print(len(df_ap_pa))
print("study_id: ", len(df_ap_pa["study_id"].unique()))
print("subject_id: ", len(df_ap_pa["subject_id"].unique()))

243324
study_id:  218131
subject_id:  63945


study_id: 218131. subject_id: 63945.

In [12]:
ap_pa_dicom_id_list = df_ap_pa["dicom_id"].unique().tolist()
print(len(ap_pa_dicom_id_list))

243324


In [13]:
selected_label_json = label_json[label_json["dicom_id"].isin(ap_pa_dicom_id_list)]
selected_label_json = selected_label_json.reset_index()
selected_label_json.head()

,index,study_id,subject_id,dicom_id,view,study_order,entity
0,0,55088298,18936629,61976388-5e534624-f6465079-76ea9caf-116f9938,antero-posterior,5,"{'pneumothorax': {'id': 0, 'entity_name': 'pne..."
1,1,51480826,18936629,ee88e702-2a3f531e-874e0bc0-e4e469e8-b34ed57a,antero-posterior,11,"{'pleural effusion': {'id': 0, 'entity_name': ..."
2,2,51268226,18936629,159014a8-5252eca6-b3c13717-20610f8d-ff5d2103,antero-posterior,24,{}
3,3,58729526,18936629,58c1653b-5d091bf0-ce12c7b3-7f6c738d-cd6f4e1b,postero-anterior,14,"{'pneumothorax': {'id': 0, 'entity_name': 'pne..."
4,4,56018384,18936629,a8becc3c-98292acf-114ca160-bb1e226a-fa031457,antero-posterior,32,"{'lung opacity': {'id': 0, 'entity_name': 'lun..."


In [14]:
print("study_id: ", len(selected_label_json["study_id"].unique()))
print("subject_id: ", len(selected_label_json["subject_id"].unique()))

study_id:  217982
subject_id:  63903


### Dict Keys

+ id: disease id.
+ entity_name: disease name
+ report_name: disease name in report
+ location: disease location
+ type: disease type
+ level: disease level
+ probability: disease uncertainty
+ probability_score: disease uncertainty score

In [15]:
# get disease name
print(selected_label_json["entity"][0].keys())
# get disease level
print(selected_label_json["entity"][0]["pneumothorax"]["level"])
# get disease location
print(selected_label_json["entity"][0]["pneumothorax"]["location"])
# get disease probability
print(selected_label_json["entity"][0]["pneumothorax"]["probability"])
# get disease probability score
print(selected_label_json["entity"][0]["pneumothorax"]["probability_score"])

dict_keys(['pneumothorax', 'atelectasis', 'edema'])
['minimal']
['left']
positive
3


### Statistic Disease Categories

There are 'pneumothorax', 'atelectasis', 'edema', 'pleural effusion', 'lung opacity', 'consolidation', 'pleural thickening', 'pneumonia', 'emphysema', 'fracture', 'tortuosity of the thoracic aorta', 'vascular congestion', 'air collection', 'cardiomegaly', 'calcification', 'enlargement of the cardiac silhouette', 'hilar congestion', 'scoliosis', 'blunting of the costophrenic angle', 'thymoma', 'heart failure', 'hernia', 'granuloma', 'hematoma', 'pneumomediastinum', 'tortuosity of the descending aorta', 'contusion', 'gastric distention', 'hypoxemia', 'hypertensive heart disease' 30 kinds of diseases。

In [16]:
# statistics disease category
disease_list = []
for i in range(len(selected_label_json)):
    a = list(selected_label_json["entity"][i].keys())
    #print(a)
    if len(a) > 0:
        for j in range(len(a)):
            if a[j] not in disease_list:
                disease_list.append(a[j])

print(disease_list)
print(len(disease_list))

['pneumothorax', 'atelectasis', 'edema', 'pleural effusion', 'lung opacity', 'consolidation', 'pleural thickening', 'pneumonia', 'emphysema', 'fracture', 'tortuosity of the thoracic aorta', 'vascular congestion', 'cardiomegaly', 'calcification', 'enlargement of the cardiac silhouette', 'hilar congestion', 'scoliosis', 'blunting of the costophrenic angle', 'heart failure', 'hernia', 'granuloma', 'hematoma', 'pneumomediastinum', 'tortuosity of the descending aorta ', 'contusion', 'air collection', 'gastric distention', 'hypoxemia', 'hypertensive heart disease', 'thymoma']
30


In [1]:
disease_categories = ['pneumothorax', 'atelectasis', 'edema', 'pleural effusion', 
                      'lung opacity', 'consolidation', 'pleural thickening', 'pneumonia', 
                      'emphysema', 'fracture', 'tortuosity of the thoracic aorta', 
                      'vascular congestion', 'cardiomegaly', 'calcification', 
                      'enlargement of the cardiac silhouette', 'hilar congestion', 
                      'scoliosis', 'blunting of the costophrenic angle', 'heart failure', 
                      'hernia', 'granuloma', 'hematoma', 'pneumomediastinum', 
                      'tortuosity of the descending aorta', 'contusion', 'air collection', 
                      'gastric distention', 'hypoxemia', 'hypertensive heart disease', 'thymoma']

disease_categories_extend = ["atelectasis", "blunting of the costophrenic angle", "calcification", "cardiomegaly", 
                             "consolidation", "edema", "emphysema", "fracture", "granuloma", "hernia", "lung opacity", 
                             "pleural effusion", "pleural thickening", "pneumonia", "pneumothorax", "scoliosis", 
                             "tortuosity of the thoracic aorta", "vascular congestion", "air collection", 
                             "enlargement of the cardiac silhouette", "heart failure", "tortuosity of the descending aorta"]

### Disease Level

In [17]:
# statistics disease severity
level_list = []
for i in range(len(selected_label_json)):
    a = list(selected_label_json["entity"][i].keys())   # diseases list
    b = selected_label_json["entity"][i]
    #print(a)
    if len(a) > 0:
        for j in range(len(a)):
            if b[a[j]]["level"] not in level_list:
                level_list.append(b[a[j]]["level"])
print(level_list[0:10])
print(len(level_list))

[['minimal'], ['mild'], None, ['small'], ['increasing'], ['moderate'], ['severe'], ['trace'], ['minimally'], ['acute']]
144


None indicate that there is no description, not that there is no disease.

In [18]:
level_filted = []
for i in range(len(level_list)):
    if level_list[i] == None:
        level_filted.append(str("no"))
    else:
        for j in range(len(level_list[i])):
            if level_list[i][j] not in level_filted:
                level_filted.append(level_list[i][j])

print(level_filted)
print(len(level_filted))

['minimal', 'mild', 'no', 'small', 'increasing', 'moderate', 'severe', 'trace', 'minimally', 'acute', 'minor', 'subtle', 'mild to moderate', 'moderate to severe', 'massive', 'moderately', 'mildly', 'moderate to large', 'decreasing']
19


disease level can be divided into several classes after the merge:

+ no
+ mild, small, trace, minor, minimal, minimally, subtle, mildly
+ increasing
+ decreasing
+ moderate, moderately
+ mild to moderate
+ moderate to severe, moderate to large
+ severe, acute, massive

In [20]:
# statistics of disease level in selected label json
disease_level_list = ([[] for i in range(len(disease_categories))])
for i in range(len(selected_label_json)):
    a = list(selected_label_json["entity"][i].keys())   # obtain disease list
    b = selected_label_json["entity"][i]
    #print(a)
    if len(a) > 0:
        for j in range(len(a)):
            if a[j] not in disease_categories_extend:
                continue
            elif a[j] == "air collection":
                merged_disease = "lung opacity"
            elif a[j] == "enlargement of the cardiac silhouette":
                merged_disease = "cardiomegaly"
            elif a[j] == "herat failure":
                merged_disease = "edema"
            elif a[j] == "tortuosity of the descending aorta":
                merged_disease = "tortuosity of the thoracic aorta"
            elif a[j] in disease_categories:
                merged_disease = a[j]              
            k = disease_categories.index(merged_disease) # disease 的 index
            if b[a[j]]["level"] not in disease_level_list[k]:
                disease_level_list[k].append(b[a[j]]["level"])
#print(disease_level_list)
print(len(disease_level_list))

30


### Disease uncertainty

In [21]:
# probability
prob_list = []
prob_score_list = []
for i in range(len(selected_label_json)):
    a = list(selected_label_json["entity"][i].keys())
    b = selected_label_json["entity"][i]
    #print(a)
    if len(a) > 0:
        for j in range(len(a)):
            if a[j] not in disease_categories:
                continue

            if "probability" in b[a[j]].keys():     
                if b[a[j]]["probability"] not in prob_list:
                    prob_list.append(b[a[j]]["probability"])
                    prob_score_list.append(b[a[j]]["probability_score"])

print(prob_list)
print(prob_score_list)
print(len(prob_list))

['positive', 'without', 'probabl', 'no ', 'likely', 'not exclude', 'may', 'could', 'negative', 'change in', 'possibl', 'difficult exclude', 'clear of', 'potential', 'might', 'lack of', 'cannot be assessed', 'impossible exclude', 'exclude', 'unlikely', 'rule out', 'not ruled out', 'cannot be identified', 'cannot be evaluated', 'not rule out', 'cannot be confirmed', 'cannot be accurately assessed', 'ruled out', 'difficult rule out']
[3, -3, 2, -3, 2, 1, 2, 2, -3, 3, 2, 1, -3, 1, 1, -3, 0, 1, -3, -2, -3, 1, 0, 0, 1, 0, 0, -3, 1]
29


In [22]:
prob_dict = dict(zip(prob_list, prob_score_list))
print(prob_dict)

{'positive': 3, 'without': -3, 'probabl': 2, 'no ': -3, 'likely': 2, 'not exclude': 1, 'may': 2, 'could': 2, 'negative': -3, 'change in': 3, 'possibl': 2, 'difficult exclude': 1, 'clear of': -3, 'potential': 1, 'might': 1, 'lack of': -3, 'cannot be assessed': 0, 'impossible exclude': 1, 'exclude': -3, 'unlikely': -2, 'rule out': -3, 'not ruled out': 1, 'cannot be identified': 0, 'cannot be evaluated': 0, 'not rule out': 1, 'cannot be confirmed': 0, 'cannot be accurately assessed': 0, 'ruled out': -3, 'difficult rule out': 1}


### Disease location

In [23]:
# Count which localization descriptions each disease has
location_list = []
for i in range(len(selected_label_json)):
    a = list(selected_label_json["entity"][i].keys())
    b = selected_label_json["entity"][i]
    #print(a)
    if len(a) > 0:
        for j in range(len(a)):
            if b[a[j]]["location"] not in location_list:
                location_list.append(b[a[j]]["location"])
print(location_list[0:10])
print(len(location_list))

[['left'], ['left', 'basal'], None, ['basilar', 'lung'], ['left', 'apical'], ['bibasilar'], ['bilateral'], ['right', 'lower', 'lung'], ['retrocardiac'], ['right']]
813


In [24]:
# Count each disease localization num
location_num_list = []
for i in range(len(selected_label_json)):
    a = list(selected_label_json["entity"][i].keys())
    b = selected_label_json["entity"][i]
    #print(a)
    if len(a) > 0:
        for j in range(len(a)):
            if b[a[j]]["location"] == None:
                loc = "None"
            else:
                loc = " ".join(b[a[j]]["location"])
            location_num_list.append(loc)
print(location_num_list[0:10])
print(len(location_num_list))

['left', 'left basal', 'None', 'left', 'basilar lung', 'None', 'left apical', 'None', 'None', 'None']
732751


In [25]:
from collections import Counter

counter = Counter(location_num_list)
# print(counter)
for key, value in counter.items():
    print(key, ":", value)

left : 19145
left basal : 1554
None : 587439
basilar lung : 39
left apical : 1758
bibasilar : 15273
bilateral : 22419
right lower lung : 4152
retrocardiac : 6128
right : 19908
right rib : 1434
bilateral lung base : 13
basal : 688
rib : 3757
lower lung : 1507
right pleural : 68
right upper lower : 5
right basilar : 2695
left lung base : 1566
left lower lung : 6924
basilar : 2150
lung base : 198
left basilar : 3509
left rib : 1303
right middle lung : 1085
middle to lower left lung : 51
lung : 3067
right apical : 2450
left retrocardiac : 2160
left lower lung right : 32
right basal left : 28
bilateral lower lung : 670
bilateral bibasilar : 25
right middle lower lung : 227
apical : 852
bilateral rib : 325
right basal : 1595
middle lung : 190
right lung base retrocardiac : 9
right upper : 111
left upper lung : 674
lung bibasilar : 217
left lower lobe : 97
right upper lung : 1606
pleural : 537
middle lung right basilar : 4
right lung : 781
lung bilateral : 79
pericardial : 839
bilateral basal

In [27]:
# Traversing the location_list extracted above, 
# because the words in the location_list are repeated.
location_filted = []
for i in range(len(location_list)):
    if location_list[i] == None:
        location_filted.append(str("None"))
    else:
        for j in range(len(location_list[i])):
            if location_list[i][j] not in location_filted:
                location_filted.append(location_list[i][j])

print(location_filted)
print(len(location_filted))

['left', 'basal', 'None', 'basilar', 'lung', 'apical', 'bibasilar', 'bilateral', 'right', 'lower', 'retrocardiac', 'rib', 'base', 'pleural', 'upper', 'middle', 'to', 'lobe', 'pericardial', 'apicolateral', 'area', 'right-sided', 'mid', 'left-sided', 'mid to lower']
25


In [28]:
disease_level_disease_list = []
for i in range(len(selected_label_json)):
    json_dict = selected_label_json["entity"].values[i]
    triple_list = []
    disease_list = json_dict.keys()
    if len(disease_list) == 0:
        #print("No finding")
        triple_list = []
    else:
        disease_list = list(disease_list)
        #print(disease_list)
        for j in range(len(disease_list)):
            if disease_list[j] not in disease_categories_extend:
                continue
            elif disease_list[j] == "air collection":
                merged_disease = "lung opacity"
            elif disease_list[j] == "enlargement of the cardiac silhouette":
                merged_disease = "cardiomegaly"
            elif disease_list[j] == "herat failure":
                merged_disease = "edema"
            elif disease_list[j] == "tortuosity of the descending aorta":
                merged_disease = "tortuosity of the thoracic aorta"
            elif disease_list[j] in disease_categories:
                merged_disease = disease_list[j]
            
            if "level" in json_dict[disease_list[j]].keys():
                disease_level = json_dict[disease_list[j]]["level"]
            if "location" in json_dict[disease_list[j]].keys():
                disease_location = json_dict[disease_list[j]]["location"]
            if "probability" in json_dict[disease_list[j]].keys():
                disease_prob = json_dict[disease_list[j]]["probability"]
            triple_list.append([disease_location, disease_level, merged_disease, disease_prob])


    for k in range(len(triple_list)):
        # disease that not exist
        if triple_list[k][-1] in ["no ", "without", "clear of", "negative", "exclude", "lack of", "rule out"]:
            disease_level_disease = "no" + " " + str(triple_list[k][2])
        # disease that doesnt have severity
        elif triple_list[k][1] == None:
            disease_level_disease = triple_list[k][2]
        # disease has severity and uncertainty
        else:
            if triple_list[k][1][0] in ["mild", "small", "trace", "minor", "minimal", "minimally", "subtle", "mildly"]:
                disease_level_merge = "mild"
            elif triple_list[k][1][0] in ["moderate", "moderately", "mild to moderate", "increasing", "decreasing"]:
                disease_level_merge = "moderate"
            elif triple_list[k][1][0] in ["severe", "acute", "massive", "moderate to severe", "moderate to large"]:
                disease_level_merge = "severe"
            disease_level_disease = disease_level_merge + " " + triple_list[k][2]
        disease_level_disease_list.append(disease_level_disease)
print(len(disease_level_disease_list))

723682


In [29]:
from collections import Counter

result = Counter(disease_level_disease_list)
print(dict(result))
print(len(list(dict(result).keys())))

{'mild pneumothorax': 3810, 'mild atelectasis': 10821, 'no edema': 31475, 'mild pleural effusion': 28559, 'lung opacity': 55733, 'atelectasis': 58755, 'no pneumothorax': 123490, 'consolidation': 14593, 'pleural thickening': 2802, 'pleural effusion': 29150, 'mild edema': 11145, 'pneumonia': 29414, 'moderate lung opacity': 1070, 'emphysema': 4324, 'pneumothorax': 5628, 'fracture': 7018, 'tortuosity of the thoracic aorta': 804, 'edema': 15125, 'vascular congestion': 6484, 'moderate cardiomegaly': 11696, 'cardiomegaly': 12957, 'no consolidation': 50602, 'no pleural effusion': 100808, 'calcification': 7021, 'no vascular congestion': 8803, 'mild lung opacity': 4478, 'mild vascular congestion': 5452, 'no pneumonia': 28037, 'scoliosis': 1673, 'severe emphysema': 627, 'mild consolidation': 231, 'mild cardiomegaly': 8301, 'mild blunting of the costophrenic angle': 587, 'no lung opacity': 5605, 'no fracture': 4796, 'hernia': 2060, 'blunting of the costophrenic angle': 2058, 'moderate atelectasis'

In [30]:
disease_level_disease_list_filtered = []
dict_2_list = list(dict(result).keys())
for i in range(len(dict_2_list)):
    num = dict(result)[dict_2_list[i]]
    if num >= 100:
        disease_level_disease_list_filtered.append(dict_2_list[i])
print(len(disease_level_disease_list_filtered))
print(disease_level_disease_list_filtered)

68
['mild pneumothorax', 'mild atelectasis', 'no edema', 'mild pleural effusion', 'lung opacity', 'atelectasis', 'no pneumothorax', 'consolidation', 'pleural thickening', 'pleural effusion', 'mild edema', 'pneumonia', 'moderate lung opacity', 'emphysema', 'pneumothorax', 'fracture', 'tortuosity of the thoracic aorta', 'edema', 'vascular congestion', 'moderate cardiomegaly', 'cardiomegaly', 'no consolidation', 'no pleural effusion', 'calcification', 'no vascular congestion', 'mild lung opacity', 'mild vascular congestion', 'no pneumonia', 'scoliosis', 'severe emphysema', 'mild consolidation', 'mild cardiomegaly', 'mild blunting of the costophrenic angle', 'no lung opacity', 'no fracture', 'hernia', 'blunting of the costophrenic angle', 'moderate atelectasis', 'moderate pleural effusion', 'severe atelectasis', 'severe cardiomegaly', 'moderate edema', 'severe lung opacity', 'no cardiomegaly', 'mild pneumonia', 'granuloma', 'moderate tortuosity of the thoracic aorta', 'mild granuloma', 'se

In [31]:
a = set(disease_level_disease_list)
print(a)
print(len(a))
b = list(a)
b.sort(reverse=False)
print(b)

{'mild tortuosity of the thoracic aorta', 'moderate lung opacity', 'severe scoliosis', 'lung opacity', 'granuloma', 'severe lung opacity', 'no edema', 'moderate pleural thickening', 'mild granuloma', 'vascular congestion', 'mild vascular congestion', 'severe cardiomegaly', 'moderate atelectasis', 'no calcification', 'severe pneumonia', 'severe pleural effusion', 'mild lung opacity', 'severe calcification', 'moderate hernia', 'no consolidation', 'moderate edema', 'no cardiomegaly', 'mild pleural thickening', 'mild pneumonia', 'mild cardiomegaly', 'atelectasis', 'no granuloma', 'no pleural thickening', 'no hernia', 'no emphysema', 'mild emphysema', 'pleural thickening', 'severe atelectasis', 'mild calcification', 'edema', 'hernia', 'moderate emphysema', 'mild edema', 'cardiomegaly', 'moderate cardiomegaly', 'severe emphysema', 'mild fracture', 'mild pleural effusion', 'severe pleural thickening', 'pleural effusion', 'emphysema', 'mild blunting of the costophrenic angle', 'mild consolidat

In [3]:
# disease_level_category = b
disease_level_category = ['atelectasis', 'mild atelectasis', 'moderate atelectasis', 'severe atelectasis',
    'blunting of the costophrenic angle', 'mild blunting of the costophrenic angle',
    'calcification', 'mild calcification', 'moderate calcification', 'severe calcification',
    'cardiomegaly', 'mild cardiomegaly', 'moderate cardiomegaly', 'severe cardiomegaly',
    'consolidation', 'mild consolidation', 'moderate consolidation', 'severe consolidation',
    'edema', 'mild edema', 'moderate edema', 'severe edema', 
    'emphysema', 'mild emphysema', 'moderate emphysema', 'severe emphysema',
    'fracture', 'mild fracture', 'moderate fracture', 'severe fracture',
    'granuloma', 'mild granuloma',
    'hernia', 'mild hernia', 'moderate hernia', 'severe hernia',
    'lung opacity', 'mild lung opacity', 'moderate lung opacity', 'severe lung opacity',
    'pleural effusion', 'mild pleural effusion', 'moderate pleural effusion', 'severe pleural effusion',         
    'pleural thickening', 'mild pleural thickening', 'moderate pleural thickening', 'severe pleural thickening',
    'pneumonia', 'mild pneumonia', 'moderate pneumonia', 'severe pneumonia',
    'pneumothorax', 'mild pneumothorax', 'moderate pneumothorax', 'severe pneumothorax',
    'scoliosis', 'mild scoliosis', 'moderate scoliosis', 'severe scoliosis',
    'tortuosity of the thoracic aorta', 'mild tortuosity of the thoracic aorta', 'moderate tortuosity of the thoracic aorta', 'severe tortuosity of the thoracic aorta',
    'vascular congestion', 'mild vascular congestion', 'moderate vascular congestion', 'severe vascular congestion',
]

In [33]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

columns = ['subject_id', 'study_id', 'dicom_id', 'path', 'report_path', 'split', 
           'Rows', 'Columns'] + disease_level_category

ap_pa_data_new = pd.DataFrame(columns=columns)
for i in range(8):
    ap_pa_data_new[columns[i]] = df_ap_pa[columns[i]]
ap_pa_data_new.head()

,subject_id,study_id,dicom_id,path,report_path,split,Rows,Columns,atelectasis,mild atelectasis,...,moderate scoliosis,severe scoliosis,tortuosity of the thoracic aorta,mild tortuosity of the thoracic aorta,moderate tortuosity of the thoracic aorta,severe tortuosity of the thoracic aorta,vascular congestion,mild vascular congestion,moderate vascular congestion,severe vascular congestion
0,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...,files/p10/p10000032/s50414267.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,files/p10/p10000032/s53189527.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,files/p10/p10000032/s53911762/68b5c4b1-227d048...,files/p10/p10000032/s53911762.txt,train,2705,2539,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,files/p10/p10000032/s53911762.txt,train,2906,2258,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,files/p10/p10000032/s56699142.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
pa_data_new_copy = ap_pa_data_new
for i in tqdm(range(len(ap_pa_data_new))):
    # 填充基本信息
    subject_id = ap_pa_data_new.loc[i, "subject_id"]
    study_id = ap_pa_data_new.loc[i, "study_id"]
    selected_line = label_json[(label_json["subject_id"] == subject_id) &
                               (label_json["study_id"] == study_id)].copy()
    # 获得 disease entity
    # 获得 disease 三元组，记录每个 disease 的 level, disease, uncertainty
    json_dict = selected_line["entity"].values[0]
    triple_list = []
    disease_list = json_dict.keys()
    if len(disease_list) == 0:
        #print("No finding")
        triple_list = []
    else:
        disease_list = list(disease_list)
        #print(disease_list)
        for j in range(len(disease_list)):
            if disease_list[j] not in disease_categories_extend:
                continue
            elif disease_list[j] == "air collection":
                merged_disease = "lung opacity"
            elif disease_list[j] == "enlargement of the cardiac silhouette":
                merged_disease = "cardiomegaly"
            elif disease_list[j] == "herat failure":
                merged_disease = "edema"
            elif disease_list[j] == "tortuosity of the descending aorta":
                merged_disease = "tortuosity of the thoracic aorta"
            elif disease_list[j] in disease_categories:
                merged_disease = disease_list[j]
            if "level" in json_dict[disease_list[j]].keys():
                disease_level = json_dict[disease_list[j]]["level"]
            if "location" in json_dict[disease_list[j]].keys():
                disease_location = json_dict[disease_list[j]]["location"]
            if "probability" in json_dict[disease_list[j]].keys():
                disease_prob = json_dict[disease_list[j]]["probability"]
            triple_list.append([disease_location, disease_level, merged_disease, disease_prob])
    #disease_no_list = list(selected_line["no_entity"])[0]
    #print(disease_certain_list)
    # 映射 disease 的 label value
    for k in range(len(triple_list)):
        # disease that not exist
        if triple_list[k][-1] in ["no ", "without", "clear of", "negative", "exclude", "lack of", "rule out"]:
            # disease_level_disease = "no" + " " + str(triple_list[k][2])
            disease_level_disease = str(triple_list[k][2])
            disease_label_value = 0.0
        # disease that doesnt have severity
        elif triple_list[k][1] == None:
            disease_level_disease = triple_list[k][2]
            if triple_list[k][-1] in ["positive", "change in"]:
                disease_label_value = 1.0
            elif triple_list[k][-1] in ["may", "likely", "could", "probabl", "potential"]:
                disease_label_value = 0.7
            elif triple_list[k][-1] in ["might", "possibl"]:
                disease_label_value = 0.5
            else:
                disease_label_value = 0.3
        # disease has severity and uncertainty
        else:
            if triple_list[k][1][0] in ["mild", "small", "trace", "minor", "minimal", "minimally", "subtle", "mildly"]:
                disease_level_merge = "mild"
            elif triple_list[k][1][0] in ["moderate", "moderately", "mild to moderate", "increasing", "decreasing"]:
                disease_level_merge = "moderate"
            elif triple_list[k][1][0] in ["severe", "acute", "massive", "moderate to severe", "moderate to large"]:
                disease_level_merge = "severe"
            disease_level_disease = disease_level_merge + " " + triple_list[k][2]
            if triple_list[k][-1] in ["positive", "change in"]:
                disease_label_value = 1.0
            elif triple_list[k][-1] in ["may", "likely", "could", "probabl", "potential"]:
                disease_label_value = 0.7
            elif triple_list[k][-1] in ["might", "possibl"]:
                disease_label_value = 0.5
            else:
                disease_label_value = 0.3
        pa_data_new_copy.loc[i, disease_level_disease] = disease_label_value

100%|██████████| 243324/243324 [19:07<00:00, 212.01it/s]


In [35]:
pa_data_new_copy.to_csv("results/ap_pa_uncertain.csv", index = None)

In [36]:
import os
import pandas as pd

df = pd.read_csv("results/ap_pa_uncertain.csv")
df.head()

,subject_id,study_id,dicom_id,path,report_path,split,Rows,Columns,atelectasis,mild atelectasis,...,moderate scoliosis,severe scoliosis,tortuosity of the thoracic aorta,mild tortuosity of the thoracic aorta,moderate tortuosity of the thoracic aorta,severe tortuosity of the thoracic aorta,vascular congestion,mild vascular congestion,moderate vascular congestion,severe vascular congestion
0,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...,files/p10/p10000032/s50414267.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,files/p10/p10000032/s53189527.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,files/p10/p10000032/s53911762/68b5c4b1-227d048...,files/p10/p10000032/s53911762.txt,train,2705,2539,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,files/p10/p10000032/s53911762.txt,train,2906,2258,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,files/p10/p10000032/s56699142.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df.columns

Index(['subject_id', 'study_id', 'dicom_id', 'path', 'report_path', 'split',
       'Rows', 'Columns', 'atelectasis', 'mild atelectasis',
       'moderate atelectasis', 'severe atelectasis',
       'blunting of the costophrenic angle',
       'mild blunting of the costophrenic angle', 'calcification',
       'mild calcification', 'moderate calcification', 'severe calcification',
       'cardiomegaly', 'mild cardiomegaly', 'moderate cardiomegaly',
       'severe cardiomegaly', 'consolidation', 'mild consolidation',
       'moderate consolidation', 'severe consolidation', 'edema', 'mild edema',
       'moderate edema', 'severe edema', 'emphysema', 'mild emphysema',
       'moderate emphysema', 'severe emphysema', 'fracture', 'mild fracture',
       'moderate fracture', 'severe fracture', 'granuloma', 'mild granuloma',
       'hernia', 'mild hernia', 'moderate hernia', 'severe hernia',
       'lung opacity', 'mild lung opacity', 'moderate lung opacity',
       'severe lung opacity', 'pleu

## Split the dataset

### Split the dataset follow the original rule

In [38]:
df_train = df[df["split"] == "train"]
print(len(df_train))

237962


In [39]:
df_val = df[df["split"] == "validate"]
print(len(df_val))

1959


In [40]:
df_test = df[df["split"] == "test"]
print(len(df_test))

3403


### Resplit the dataset

split the dataset randomly.

In [10]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("results/ap_pa_uncertain.csv")
df.head()

,subject_id,study_id,dicom_id,path,report_path,split,Rows,Columns,atelectasis,mild atelectasis,...,moderate scoliosis,severe scoliosis,tortuosity of the thoracic aorta,mild tortuosity of the thoracic aorta,moderate tortuosity of the thoracic aorta,severe tortuosity of the thoracic aorta,vascular congestion,mild vascular congestion,moderate vascular congestion,severe vascular congestion
0,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...,files/p10/p10000032/s50414267.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,files/p10/p10000032/s53189527.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,files/p10/p10000032/s53911762/68b5c4b1-227d048...,files/p10/p10000032/s53911762.txt,train,2705,2539,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,files/p10/p10000032/s53911762.txt,train,2906,2258,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,files/p10/p10000032/s56699142.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_train, df_val_test = train_test_split(df,  train_size=0.8, random_state=42)
print(len(df_train))

194659


In [45]:
df_val, df_test = train_test_split(df_val_test, train_size=0.5, random_state=42)
print(len(df_val))
print(len(df_test))

24332
24333


In [46]:
df_train.to_csv("results/ap_pa_uncertain_random_train.csv", index = None)
df_val.to_csv("results/ap_pa_uncertain_random_val.csv", index = None)
df_test.to_csv("results/ap_pa_uncertain_random_test.csv", index = None)

In [1]:
import os
import pandas as pd

df_train = pd.read_csv("results/ap_pa_uncertain_random_train.csv")
df_train.head()

,subject_id,study_id,dicom_id,path,report_path,split,Rows,Columns,atelectasis,mild atelectasis,...,moderate scoliosis,severe scoliosis,tortuosity of the thoracic aorta,mild tortuosity of the thoracic aorta,moderate tortuosity of the thoracic aorta,severe tortuosity of the thoracic aorta,vascular congestion,mild vascular congestion,moderate vascular congestion,severe vascular congestion
0,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...,files/p10/p10000032/s50414267.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,files/p10/p10000032/s53189527.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,files/p10/p10000032/s53911762/68b5c4b1-227d048...,files/p10/p10000032/s53911762.txt,train,2705,2539,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,files/p10/p10000032/s53911762.txt,train,2906,2258,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,files/p10/p10000032/s56699142.txt,train,3056,2544,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_a = pd.DataFrame(columns=[1.0, 0.7, 0.5, 0.3, 0.0], index=disease_level_category)
df_a

,1.0,0.7,0.5,0.3,0.0
atelectasis,NaN,NaN,NaN,NaN,NaN
mild atelectasis,NaN,NaN,NaN,NaN,NaN
moderate atelectasis,NaN,NaN,NaN,NaN,NaN
severe atelectasis,NaN,NaN,NaN,NaN,NaN
blunting of the costophrenic angle,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
severe tortuosity of the thoracic aorta,NaN,NaN,NaN,NaN,NaN
vascular congestion,NaN,NaN,NaN,NaN,NaN
mild vascular congestion,NaN,NaN,NaN,NaN,NaN
moderate vascular congestion,NaN,NaN,NaN,NaN,NaN


In [32]:
for i in range(len(disease_level_category)):
    a = df[disease_level_category[i]].value_counts() # count df
    a = a.sort_index(ascending=False)
    a = pd.DataFrame(a).T
#print(a)
    for j in a.columns:
        df_a.loc[disease_level_category[i], j] = a.loc["count", j]

In [33]:
a.loc["count", 1]

25

In [34]:
df_a

,1.0,0.7,0.5,0.3,0.0
atelectasis,45932,16153,726,2083,1284
mild atelectasis,11321,519,75,32,NaN
moderate atelectasis,1485,91,2,19,NaN
severe atelectasis,590,48,NaN,2,NaN
blunting of the costophrenic angle,2163,30,8,76,44
...,...,...,...,...,...
severe tortuosity of the thoracic aorta,10,NaN,NaN,NaN,NaN
vascular congestion,6793,315,36,42,9770
mild vascular congestion,5562,287,192,33,NaN
moderate vascular congestion,446,10,7,NaN,NaN


In [55]:
df_a.to_csv("results/uncertain_distribution.csv")

In [30]:
df_a = pd.DataFrame(columns=[1.0, 0.7, 0.5, 0.3, 0.0], index=disease_level_category)
for i in range(len(disease_level_category)):
    a = df_train[disease_level_category[i]].value_counts() # count df_train
    a = a.sort_index(ascending=False)
    a = pd.DataFrame(a).T
    for j in a.columns:
        df_a.loc[disease_level_category[i], j] = a.loc["count", j]

In [31]:
df_a

,1.0,0.7,0.5,0.3,0.0
atelectasis,45932,16153,726,2083,1284
mild atelectasis,11321,519,75,32,NaN
moderate atelectasis,1485,91,2,19,NaN
severe atelectasis,590,48,NaN,2,NaN
blunting of the costophrenic angle,2163,30,8,76,44
...,...,...,...,...,...
severe tortuosity of the thoracic aorta,10,NaN,NaN,NaN,NaN
vascular congestion,6793,315,36,42,9770
mild vascular congestion,5562,287,192,33,NaN
moderate vascular congestion,446,10,7,NaN,NaN


In [59]:
df_a.to_csv("results/uncertain_random_train_distribution.csv")

split the dataset by patient.

In [2]:
print(len(df))
print(len(df["subject_id"].unique()))

243324
63945


63945 patients contribute 243324 images.

In [3]:
unique_patient_ids = df["subject_id"].unique()

In [6]:
train_ids, temp_ids = train_test_split(unique_patient_ids, test_size=0.2, random_state=42)
valid_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

In [7]:
train_set = df[df['subject_id'].isin(train_ids)]
valid_set = df[df['subject_id'].isin(valid_ids)]
test_set = df[df['subject_id'].isin(test_ids)]

In [8]:
print(len(train_set))
print(len(valid_set))
print(len(test_set))

194864
24209
24251


In [9]:
train_patient = set(train_set["subject_id"])
valid_patient = set(valid_set["subject_id"])
test_patient = set(test_set["subject_id"])

if len(train_patient.intersection(valid_patient)) > 0:
    print("Repeat patient ID in two dataset.")
else:
    print("No repeat patient ID in two dataset.")

No repeat patient ID in two dataset.


In [10]:
train_set.to_csv("results/ap_pa_uncertain_patient_train.csv", index=False)
valid_set.to_csv("results/ap_pa_uncertain_patient_valid.csv", index=False)
test_set.to_csv("results/ap_pa_uncertain_patient_test.csv", index=False)